In [1]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
from selenium import webdriver
import os

In [9]:
url = 'https://www.streetcheck.co.uk/postcode/startingwith/e17'

In [3]:
# r = requests.get(url)

In [4]:
# r.status_code
# returns 403 error which means The HTTP 403 Forbidden client error status response code indicates that the server understood the request but refuses to authorize it. 

In [5]:
# os.path.abspath(os.curdir)

In [6]:
# os.path.dirname(os.path.abspath(os.curdir))

In [7]:
# os.listdir(os.path.dirname(os.path.abspath(os.curdir)))

In [4]:
# want to iterate through several root files if to find  
# Development folder which has the chrome driver in it 

is_folder = False
# dir_list = None
start = os.path.dirname(os.path.abspath(os.curdir))
chrome_driver_path= None
while not is_folder:
    if 'Development' in os.listdir(start):
        filepath=start
        is_folder=True
#         print(start)
#         print(os.listdir(start))
#         print(os.path.join(start,'Development'))
        chrome_driver_path = os.path.join(start,'Development', 'chromedriver')
#         print(chrome_driver_path)
        if 'chromedriver' in os.listdir(os.path.dirname(chrome_driver_path)):
            print('chrome driver found')        
    else:
        start=os.path.dirname(start)

chrome driver found


In [77]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)

In [78]:
driver.get(url)

In [79]:
addresses = driver.find_element_by_id("main_inner")

In [37]:
address_list = []
for i in addresses.find_elements_by_tag_name('a'):
    address_list.append((i.text, i.get_attribute('href')))
#     print(i.text)
#     print(i.get_attribute('href'))

In [76]:
# address_list[:10]

In [81]:
# TODO: Find a way to get all the text that shows "no longer in use"? might be able to 
# eliminate old addresses by using the postcodes.io API
for i in addresses.find_elements_by_tag_name('p'):
    print(i.text)
    for j in addresses.find_elements_by_tag_name('a'):
        print( j.text, j.get_attribute('href'))
#     address_list.append((i.text, i.get_attribute('href')))
#     print(i.text)
#     print(i.get_attribute('href'))

Below are all of the active postcodes that we currently have on file for the E17 postcode district. The list also includes expired postcodes that we may have data for (those that have been discontinued by Royal Mail). We maintain several years of data on expired codes. Click here to view demographic information about E17 as a whole.
Postcode Information https://www.streetcheck.co.uk/
All Postcode Districts https://www.streetcheck.co.uk/postcode/alldistricts
E17 https://www.streetcheck.co.uk/postcode/startingwith/e17
Click here to view demographic information about E17 as a whole https://www.streetcheck.co.uk/postcodedistrict/e17
London, E17 0AA https://www.streetcheck.co.uk/postcode/e170aa
London, E17 0AB https://www.streetcheck.co.uk/postcode/e170ab
London, E17 0AD https://www.streetcheck.co.uk/postcode/e170ad
London, E17 0AE https://www.streetcheck.co.uk/postcode/e170ae
London, E17 0AF https://www.streetcheck.co.uk/postcode/e170af
London, E17 0AG https://www.streetcheck.co.uk/postcod

Waltham Forest, London, E17 0HQ https://www.streetcheck.co.uk/postcode/e170hq
London, E17 0HR https://www.streetcheck.co.uk/postcode/e170hr
London, E17 0HS https://www.streetcheck.co.uk/postcode/e170hs
London, E17 0HT https://www.streetcheck.co.uk/postcode/e170ht
London, E17 0HU https://www.streetcheck.co.uk/postcode/e170hu
London, E17 0HW https://www.streetcheck.co.uk/postcode/e170hw
London, E17 0HX https://www.streetcheck.co.uk/postcode/e170hx
London, E17 0HY https://www.streetcheck.co.uk/postcode/e170hy
London, E17 0HZ https://www.streetcheck.co.uk/postcode/e170hz
London, E17 0JA https://www.streetcheck.co.uk/postcode/e170ja
London, E17 0JB https://www.streetcheck.co.uk/postcode/e170jb
London, E17 0JD https://www.streetcheck.co.uk/postcode/e170jd
London, E17 0JF https://www.streetcheck.co.uk/postcode/e170jf
London, E17 0JG https://www.streetcheck.co.uk/postcode/e170jg
London, E17 0JH https://www.streetcheck.co.uk/postcode/e170jh
London, E17 0JJ https://www.streetcheck.co.uk/postcode

London, E17 0SS https://www.streetcheck.co.uk/postcode/e170ss
London, E17 0ST https://www.streetcheck.co.uk/postcode/e170st
E17 0SU https://www.streetcheck.co.uk/postcode/e170su
London, E17 0SW https://www.streetcheck.co.uk/postcode/e170sw
London, E17 0SX https://www.streetcheck.co.uk/postcode/e170sx
London, E17 0SY https://www.streetcheck.co.uk/postcode/e170sy
London, E17 0TA https://www.streetcheck.co.uk/postcode/e170ta
London, E17 0TB https://www.streetcheck.co.uk/postcode/e170tb
London, E17 0TD https://www.streetcheck.co.uk/postcode/e170td
London, E17 0TE https://www.streetcheck.co.uk/postcode/e170te
London, E17 0TF https://www.streetcheck.co.uk/postcode/e170tf
Waltham Forest, London, E17 0TG https://www.streetcheck.co.uk/postcode/e170tg
London, E17 0TH https://www.streetcheck.co.uk/postcode/e170th
London, E17 0TJ https://www.streetcheck.co.uk/postcode/e170tj
Waltham Forest, London, E17 0TL https://www.streetcheck.co.uk/postcode/e170tl
Waltham Forest, London, E17 0TN https://www.st

Forest Road, Waltham Forest, London, E17 3ED https://www.streetcheck.co.uk/postcode/e173ed
Forest Road, Waltham Forest, London, E17 3EE https://www.streetcheck.co.uk/postcode/e173ee
Forest Road, Waltham Forest, London, E17 3EF https://www.streetcheck.co.uk/postcode/e173ef
Attlee Terrace, Prospect Hill, Waltham Forest, London, E17 3EG https://www.streetcheck.co.uk/postcode/e173eg
Attlee Terrace, Prospect Hill, Waltham Forest, London, E17 3EH https://www.streetcheck.co.uk/postcode/e173eh
The Lindens, Prospect Hill, Waltham Forest, London, E17 3EJ https://www.streetcheck.co.uk/postcode/e173ej
Prospect Hill, Waltham Forest, London, E17 3EL https://www.streetcheck.co.uk/postcode/e173el
Prospect Hill, Waltham Forest, London, E17 3EN https://www.streetcheck.co.uk/postcode/e173en
Prospect Hill, Waltham Forest, London, E17 3EP https://www.streetcheck.co.uk/postcode/e173ep
Attlee Terrace, Prospect Hill, Waltham Forest, London, E17 3EQ https://www.streetcheck.co.uk/postcode/e173eq
Prospect Hill, 

Woodlands Road, Waltham Forest, London, E17 3LE https://www.streetcheck.co.uk/postcode/e173le
Woodlands Road, Waltham Forest, London, E17 3LF https://www.streetcheck.co.uk/postcode/e173lf
Waverley Road, Waltham Forest, London, E17 3LG https://www.streetcheck.co.uk/postcode/e173lg
Waverley Avenue, Waltham Forest, London, E17 3LH https://www.streetcheck.co.uk/postcode/e173lh
Stocksfield Road, Waltham Forest, London, E17 3LJ https://www.streetcheck.co.uk/postcode/e173lj
Wood Street, Waltham Forest, London, E17 3LL https://www.streetcheck.co.uk/postcode/e173ll
Stocksfield Road, Waltham Forest, London, E17 3LN https://www.streetcheck.co.uk/postcode/e173ln
Cuthbert Road, Waltham Forest, London, E17 3LP https://www.streetcheck.co.uk/postcode/e173lp
Waverley Road, Waltham Forest, London, E17 3LQ https://www.streetcheck.co.uk/postcode/e173lq
Stocksfield Road, Waltham Forest, London, E17 3LR https://www.streetcheck.co.uk/postcode/e173lr
Linford Road, Waltham Forest, London, E17 3LS https://www.s

...

William Morris Close, Waltham Forest, London, E17 5JU https://www.streetcheck.co.uk/postcode/e175ju
Winns Avenue, Lloyd Park, Waltham Forest, London, E17 5JW https://www.streetcheck.co.uk/postcode/e175jw
Holmes Avenue, Waltham Forest, London, E17 5JX https://www.streetcheck.co.uk/postcode/e175jx
Bemsted Road, Waltham Forest, London, E17 5JY https://www.streetcheck.co.uk/postcode/e175jy
Bemsted Road, Waltham Forest, London, E17 5JZ https://www.streetcheck.co.uk/postcode/e175jz
Mersey Road, Waltham Forest, London, E17 5LA https://www.streetcheck.co.uk/postcode/e175la
Mersey Road, Waltham Forest, London, E17 5LB https://www.streetcheck.co.uk/postcode/e175lb
Forest Road, Waltham Forest, London, E17 5LD https://www.streetcheck.co.uk/postcode/e175ld
Diana Road, Waltham Forest, London, E17 5LE https://www.streetcheck.co.uk/postcode/e175le
Diana Road, Waltham Forest, London, E17 5LF https://www.streetcheck.co.uk/postcode/e175lf
Badlis Road, Waltham Forest, London, E17 5LG https://www.streetche

Higham Hill Road, Waltham Forest, London, E17 5RX https://www.streetcheck.co.uk/postcode/e175rx
Manor Road, Waltham Forest, London, E17 5RY https://www.streetcheck.co.uk/postcode/e175ry
Manor Road, Waltham Forest, London, E17 5RZ https://www.streetcheck.co.uk/postcode/e175rz
Higham Hill Road, Waltham Forest, London, E17 5SA https://www.streetcheck.co.uk/postcode/e175sa
Manor Road, Waltham Forest, London, E17 5SB https://www.streetcheck.co.uk/postcode/e175sb
Uplands Business Park, Blackhorse Lane, London, E17 5SD https://www.streetcheck.co.uk/postcode/e175sd
London, E17 5SE https://www.streetcheck.co.uk/postcode/e175se
London, E17 5SH https://www.streetcheck.co.uk/postcode/e175sh
London, E17 5SJ https://www.streetcheck.co.uk/postcode/e175sj
London, E17 5SL https://www.streetcheck.co.uk/postcode/e175sl
London, E17 5SN https://www.streetcheck.co.uk/postcode/e175sn
Sandpiper Close, Waltham Forest, London, E17 5SP https://www.streetcheck.co.uk/postcode/e175sp
Cormorant Close, Waltham Forest

London, E17 6AU https://www.streetcheck.co.uk/postcode/e176au
Goldsmith Road, Waltham Forest, London, E17 6AW https://www.streetcheck.co.uk/postcode/e176aw
St. Andrew's Road, Waltham Forest, London, E17 6AX https://www.streetcheck.co.uk/postcode/e176ax
Chamberlain Place, Waltham Forest, London, E17 6AY https://www.streetcheck.co.uk/postcode/e176ay
Chamberlain Place, Waltham Forest, London, E17 6AZ https://www.streetcheck.co.uk/postcode/e176az
St. Andrew's Road, Waltham Forest, London, E17 6BA https://www.streetcheck.co.uk/postcode/e176ba
St. Andrew's Road, Waltham Forest, London, E17 6BB https://www.streetcheck.co.uk/postcode/e176bb
St. Andrew's Road, Waltham Forest, London, E17 6BD https://www.streetcheck.co.uk/postcode/e176bd
London, E17 6BE https://www.streetcheck.co.uk/postcode/e176be
Pasquier Road, Waltham Forest, London, E17 6BF https://www.streetcheck.co.uk/postcode/e176bf
London, E17 6BG https://www.streetcheck.co.uk/postcode/e176bg
Sutherland Road, Waltham Forest, London, E17 

Ferry Lane Industrial Estate, Wickford Way, London, E17 6HG https://www.streetcheck.co.uk/postcode/e176hg
Ferry Lane Industrial Estate, Waltham Forest, London, E17 6HH https://www.streetcheck.co.uk/postcode/e176hh
Blackhorse Lane, Waltham Forest, London, E17 6HJ https://www.streetcheck.co.uk/postcode/e176hj
Clifton Avenue, Waltham Forest, London, E17 6HL https://www.streetcheck.co.uk/postcode/e176hl
Pembar Avenue, Waltham Forest, London, E17 6HN https://www.streetcheck.co.uk/postcode/e176hn
Tavistock Avenue, Waltham Forest, London, E17 6HP https://www.streetcheck.co.uk/postcode/e176hp
London, E17 6HQ https://www.streetcheck.co.uk/postcode/e176hq
Tavistock Avenue, Waltham Forest, London, E17 6HR https://www.streetcheck.co.uk/postcode/e176hr
Blenheim Road, Waltham Forest, London, E17 6HS https://www.streetcheck.co.uk/postcode/e176hs
Blenheim Road, Waltham Forest, London, E17 6HT https://www.streetcheck.co.uk/postcode/e176ht
Queen Elizabeth Road, Waltham Forest, London, E17 6HU https://ww

KeyboardInterrupt: 

In [82]:
# driver.close() # this will close a specific tab
driver.quit() # this will close the entire browser